In [ ]:
#import geopandas as gpd
import matplotlib.pyplot as plt
#import pyproj
import pandas as pd
import h5py    
import os
from shapely.geometry import Point, Polygon, MultiPolygon
import numpy as np
#import elevation_aware_ssl.preprocessing as DP
from shapely.geometry import Polygon
from osgeo import osr, gdal, ogr
import torch
import geopandas as gpd
from elevation_aware_ssl import preprocessing as dp
import pickle
import tempfile
from tqdm import tqdm
from elevation_aware_ssl import EDA
import os
from IPython.display import clear_output
import re

In [ ]:
letters="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
letters_map=[{l:i for i, l in enumerate(letters)}][0]

In [ ]:
def get_patch_shapefile(patch_name, path_to_folders, frontera, unique_labels):

    coor = []
    for chip in metadata.query(f"patch=='{patch_name}'").image:

        path_to_chip_metadata = path_to_folders + "/" + chip + "/metadata.pkl"
        corners = pd.read_pickle(path_to_chip_metadata)["corners"]
        nw_x, nw_y = corners["nw"][::-1]
        se_x, se_y = corners["se"][::-1]
        coor.append([nw_x, nw_y, se_x, se_y])

    nw_x = np.array(coor)[:, 0].min()
    nw_y = np.array(coor)[:, 1].max()
    se_x = np.array(coor)[:, 2].max()
    se_y = np.array(coor)[:, 3].min()
    metadata_coor = {"corners": {"nw": np.array([nw_y, nw_x]), "se": np.array([se_y, se_x])}}

    with tempfile.NamedTemporaryFile("wb") as temp:
        pickle.dump(metadata_coor, temp)
        temp.flush()
        patch_reference = patch_name
        gdf_chip = dp.create_shapefiel_from_polygons(temp.name, patch_reference, chip_padding=0.001, crs="epsg:3116")


    gdf_intersection = dp.polygons_intersection(frontera, gdf_chip, unique_labels, patch_reference)
    gdf_intersection.rename({"labels": "elemento"}, axis=1, inplace=True)

    return gdf_intersection

In [ ]:
def create_dataset(metadata, frontera, path_temp_file, path_to_save_masks):

    images = []
    masks = []
    root = []
    id = []
    geometry_intersection = []
    current_patch = ""
    unique_labels = frontera["elemento"].unique()
    unique_labels.sort()
    print(unique_labels)
    # metadata.sort_values(by="patch", inplace=True)

    for chip_metadata, patch_name, idx in zip(tqdm(metadata.image), metadata.patch, metadata.index):

        chip_reference = chip_metadata.split("/")[1]

        # Create square polygons using chip metada
        gdf_chip = dp.create_shapefiel_from_polygons(
            f"{path_to_folders}/{chip_metadata}/metadata.pkl",
            chip_reference,
            chip_padding=0.0005,
            crs="epsg:3116",
        )

        # get patch shapefile
        if current_patch != patch_name:
            current_patch = patch_name
            patch_shapefile = get_patch_shapefile(patch_name, path_to_folders, frontera, unique_labels)

        # Intersecton between square polygons and SIPRA dataset
        gdf_intersection = dp.polygons_intersection(patch_shapefile, gdf_chip, unique_labels, chip_reference)


        if gdf_intersection.geometry.is_empty.sum() != 3:

            img = EDA.read_numpy_image(f"{path_to_data}/Dataset/{chip_metadata}/chip.npy")
            dp.from_array_to_geotiff(
                f"{path_to_data}/sentinel_geo_images/{chip_metadata}/chip.tif",
                img[:3],
                f"{path_to_data}/Dataset/{chip_metadata}/metadata.pkl",
                crs=3116,
            )

            gdf_intersection.to_file(path_temp_file + "/temp.geojson", driver="GeoJSON")

            # create labels masks
            dp.shapefiel_to_geotiff(path_temp_file + "/temp.geojson", f"temp/{chip_reference}.tif", 10, "labels_num", no_data_value=999)

            if not os.path.isdir(path_to_save_masks + f"/{chip_metadata}"):
                os.makedirs(path_to_save_masks + f"/{chip_metadata}")

            dp.crop_geotiff_chip(
                f"{path_to_folders}/{chip_metadata}/metadata.pkl",
                f"temp/{chip_reference}.tif",
                path_to_save_masks + f"/{chip_metadata}/mask.tif",
            )

            geometry_intersection.append(gdf_intersection)
            masks.append(f"{chip_metadata}/mask.tif")
            images.append(f"{chip_metadata}/chip.npz")

            id_number = re.sub(r"[a-zA-Z_\(\)\,\\\/\s+]", "", chip_metadata)
            id_letter = re.sub(r"[\d_\(\)\,\\\/\s+]", "", chip_metadata)
            id_letter = "".join(str(letters_map[l]) for l in id_letter[:3])

            id.append(int(id_number + id_letter))
            metadata = pd.DataFrame({"Id": id, "Image": images, "Mask": masks})

            # clear_output()

    gdf_intersection = pd.concat(geometry_intersection, ignore_index=True)
    gdf_intersection.crs = "epsg:3116"
    gdf_intersection.to_file("LabelGeoTiff/labels.shp")

    return gdf_intersection, metadata

In [ ]:
path_to_data = "/media/omar/storage/gdrive/Maestria/Datasets" 
metadata = pd.read_csv("/media/omar/storage/gdrive/Maestria/Theses/Preprocessing/metadata_raw_images.csv")
#metadata = metadata.drop(labels=[52809, 52810, 58697, 58700], axis=0)
metadata

In [ ]:
#path_to_frontera =f"{path_to_data}/Frontera_Agricola_Nov2021/Frontera_Agricola_Nov2021.shp"
#frontera = gpd.read_file(path_to_frontera)

In [ ]:
frontera.head()

In [ ]:
path_to_folders = f"{path_to_data}/Dataset"
path_temp_file = f"{path_to_data}/temp"
path_to_save_masks = f"{path_to_data}/LabelsGeoTiffv2"

In [ ]:
gdf, new_metadata = create_dataset(metadata.query("patch == '18NTP'").iloc[50:60].copy(), frontera, path_temp_file, path_to_save_masks)

In [ ]:
class_names = ["non_agricultural_area", "legal_exclusions", "agricultural_frontier"]

In [ ]:
#print(new_metadata.shape)
#new_metadata.to_csv("/media/omar/storage/gdrive/Maestria/Datasets/GeoDataset/LabelsGeoTiffv2/metadata.csv", index=False)
#new_metadata.head()

In [ ]:
old_metadata = pd.read_csv(f"{path_to_data}/metadata/metadata.csv")
old_metadata = old_metadata.query("Patch == '18NTN'")
old_metadata.head()

In [ ]:
path_to_label = f"{path_to_data}/LabelsGeoTiffv2/"
path_to_images = f"{path_to_data}/Sentinel_2_Images/"
_ = EDA.visualize_images_and_masks(path_to_label, path_to_images, new_metadata.head(10), temporal_dim=False, n=7, figsize=(20, 7), class_names=class_names)

In [ ]:
path_to_label = f"{path_to_data}/"
path_to_images = f"{path_to_data}/Sentinel_2_Images/"
_ = EDA.visualize_images_and_masks(path_to_label, path_to_images, old_metadata.head(10), temporal_dim=False, n=7, figsize=(20, 7), class_names=class_names)